<h1> 2. Creating a sampled dataset - HY Working</h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-3f90a9b55ca0127c'
PROJECT = 'qwiklabs-gcp-3f90a9b55ca0127c'
REGION = 'australia-southeast1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [4]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use MOD() on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [5]:
count_sql = "select count(*) from (" + query + ") where MOD(hashmonth, 4) = 1"
print(bq.Query(count_sql).execute().result().to_dataframe())

       f0_
0  9134316


In [6]:
# check how hashomonth works:
count_sql = "select count(*),  MOD(hashmonth,20) from (" + query +") where mod(hashmonth,20) < 14 group by MOD(hashmonth, 20)" 
print(bq.Query(count_sql).execute().result().to_dataframe())

        f0_  f1_
0   2120329    3
1    660523    1
2   2328931    4
3   2060460    8
4   1343107    7
5   2484033   13
6   2080569   12
7   2480582    5
8   2368609    0
9    354450   11
10   988441    2
11  1764733    9
12  1075744    6
13  1726094   10


In [7]:
trainsql = "select * from (" + query + ") where MOD(hashmonth, 10) < 9 AND RAND() < 0.0005" # 90% train
traindat = bq.Query(trainsql).execute().result().to_dataframe()

In [8]:
print(traindat.count())
traindat.head()

weight_pounds      14928
is_male            14940
mother_age         14940
plurality          14940
gestation_weeks    14849
hashmonth          14940
dtype: int64


,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.876218,True,19,1,39.0,774501970389208065
1,5.937049,False,28,1,40.0,774501970389208065
2,7.374463,True,31,1,38.0,774501970389208065
3,5.562263,True,21,1,38.0,774501970389208065
4,7.524377,False,21,1,41.0,774501970389208065


In [9]:
testsql = "select * from (" + query + ") where MOD(hashmonth, 10) = 9 AND RAND() < 0.0005" # 10% test
testdat = bq.Query(testsql).execute().result().to_dataframe()
print(testdat.count())
testdat.head()

weight_pounds      1537
is_male            1539
mother_age         1539
plurality          1539
gestation_weeks    1529
hashmonth          1539
dtype: int64


,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.625790,True,35,1,38.0,7146494315947640619
1,7.639017,False,29,1,40.0,7146494315947640619
2,7.312733,True,23,1,40.0,8904940584331855459
3,7.310529,True,24,1,31.0,260598435387740869
4,7.125340,True,26,1,37.0,5742197815970064689


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hints: <p>
Filtering:
<pre>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre
>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

In [10]:
#Filtering - clean out 'bad data'
# Let's look at a small sample of the training data
# check for number of nulls
traindf = traindat
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,14928.000000,14940.000000,14940.000000,14849.000000,1.494000e+04
mean,7.250985,27.412651,1.037149,38.611758,4.384628e+18
std,1.310352,6.168924,0.201803,2.569099,2.769932e+18
min,0.518086,13.000000,1.000000,18.000000,7.493147e+16
25%,6.563162,23.000000,1.000000,38.000000,1.639186e+18
50%,7.357928,27.000000,1.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,40.000000,6.910175e+18
max,12.125424,50.000000,4.000000,47.000000,9.183606e+18


In [11]:
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound, i.e. don't know the gender
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)' # don't know how many if more than 1
  nous['is_male'] = 'Unknown' # don't know gender
  
  return pd.concat([df, nous])

In [12]:
# Let's see a small sample of the training data now after our preprocessing
traindf_clean = preprocess(traindf)
testdf = testdat
testdf_clean = preprocess(testdf)
traindf_clean.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.876218,True,19,Single(1),39.0,774501970389208065
1,5.937049,False,28,Single(1),40.0,774501970389208065
2,7.374463,True,31,Single(1),38.0,774501970389208065
3,5.562263,True,21,Single(1),38.0,774501970389208065
4,7.524377,False,21,Single(1),41.0,774501970389208065


In [13]:
print(traindf_clean.describe(include='all'))

        weight_pounds  is_male    mother_age  plurality  gestation_weeks  \
count    29684.000000    29684  29684.000000      29684     29684.000000   
unique            NaN        3           NaN          5              NaN   
top               NaN  Unknown           NaN  Single(1)              NaN   
freq              NaN    14842           NaN      28658              NaN   
mean         7.252059      NaN     27.411131        NaN        38.615011   
std          1.306860      NaN      6.167967        NaN         2.557352   
min          0.518086      NaN     13.000000        NaN        18.000000   
25%          6.563162      NaN     23.000000        NaN        38.000000   
50%          7.354621      NaN     27.000000        NaN        39.000000   
75%          8.062305      NaN     32.000000        NaN        40.000000   
max         12.125424      NaN     50.000000        NaN        47.000000   

           hashmonth  
count   2.968400e+04  
unique           NaN  
top              N

## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [14]:
traindf_clean.to_csv('train.csv', index=False, header=False)
testdf_clean.to_csv('eval.csv', index=False, header=False)

In [15]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   3058 eval.csv
  29684 train.csv
  32742 total
==> eval.csv <==
7.62578964258,True,35,Single(1),38.0,7146494315947640619
7.6390173783,False,29,Single(1),40.0,7146494315947640619
7.31273323054,True,23,Single(1),40.0,8904940584331855459
7.31052860792,True,24,Single(1),31.0,260598435387740869
7.12534030784,True,26,Single(1),37.0,5742197815970064689
6.6910296517,False,38,Single(1),39.0,2995620979373137889
6.02082437522,True,27,Single(1),37.0,260598435387740869
8.1901730333,False,33,Single(1),41.0,7146494315947640619
6.8012607827,False,24,Single(1),42.0,5742197815970064689
7.0217230447,False,25,Single(1),40.0,5742197815970064689

==> train.csv <==
6.87621795178,True,19,Single(1),39.0,774501970389208065
5.93704871566,False,28,Single(1),40.0,774501970389208065
7.3744626639,True,31,Single(1),38.0,774501970389208065
5.56226287026,True,21,Single(1),38.0,774501970389208065
7.52437700206,False,21,Single(1),41.0,774501970389208065
5.8753192823,False,32,Single(1),39.0,774501970389208065
7.50012615

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License